Some basic work to combine synteny calls.

In [3]:
import networkx as nx


In [4]:
f = open( '/home/ndh0004/Documents/cor_exp/SS7/raw_links_awk_parsed_dc.links')
rlinks = f.read().rstrip('\n').split('\n')
f.close()

In [5]:
synGraph = nx.Graph()


Now let's make a set of unique connections

In [6]:
counter = 0
for line in rlinks :
    per_id, \
    a_gene, \
    b_gene, \
    a_con, \
    a_start, \
    a_stop, \
    b_con, \
    b_start, \
    b_stop = line.split(" ")
    target_contigs = ['Super-Scaffold_7', 'Super-Scaffold_27',
        'Super-Scaffold_453','Super-Scaffold_70']
    if a_con in target_contigs and b_con in target_contigs:
        if synGraph.has_edge(a_gene,b_gene) is False :
            synGraph.add_edge(a_gene,b_gene)
            synGraph.add_edge(b_gene,a_gene)
            synGraph.node[a_gene]['contig'] = a_con
            synGraph.node[a_gene]['start'] = a_start
            synGraph.node[b_gene]['contig'] = b_con
            synGraph.node[b_gene]['start'] = b_start
            counter +=1 
print(counter)

1568


Now that we have a graph connecting all these components. Let's add values to the nodes. For counts.

In [7]:
import pandas as pd


In [8]:
df = pd.read_csv('/home/ndh0004/Documents/cor_exp/raw_total_counts/gene.counts'
                 , sep=' ')


In [9]:

counter = 0 
for index, row in df.iterrows():
    node_q = row['gene']+'-mRNA-1'
    count_sum = sum(row[2:]) 
    count_max = max(row[2:])
    count_zeros = list(row[2:]).count(0)
    total_samples = len(row[2:])
    if synGraph.has_node(node_q):
        synGraph.node[node_q]['sum'] = count_sum
        synGraph.node[node_q]['max'] = count_max
        synGraph.node[node_q]['zeros'] = count_zeros
        synGraph.node[node_q]['total_num_counts'] = total_samples
        counter += 1
print('{c} counts added to nodes'.format(c=counter))

1686 counts added to nodes


Counts have been added to the network. Now it's time to output the results


In [10]:
from collections import Counter
lengthList= []
for miniGraph in nx.connected_components(synGraph):
    lengthList.append(len(miniGraph))
summary = Counter(lengthList)
print(summary)

Counter({3: 479, 2: 116, 4: 3, 5: 1})


In [18]:
passed = 0
failed = 0
fout = open('/home/ndh0004/Documents/cor_exp/SS7/linked_exp_python/'
            'double_linkedJan17.csv', 'w')
fout_single = open('/home/ndh0004/Documents/cor_exp/SS7/linked_exp_python/'
            'single_linkedJan17.csv', 'w')
fout.write('Agene,Astart,Asum,Acontig,Amax,Azeros,Atotal_num_counts,')
fout.write('AAgene,AAstart,AAsum,AAcontig,AAmax,AAzeros,AAtotal_num_counts,')
fout.write('Bgene,Bstart,Bsum,Bcontig,Bmax,Bzeros,Btotal_num_counts\n')

fout_single.write('Agene,Astart,Asum,Acontig,Amax,Azeros,Atotal_num_counts,')
fout_single.write('Bgene,Bstart,Bsum,Bcontig,Bmax,Bzeros,Btotal_num_counts\n')
for miniSet in nx.connected_components(synGraph):
    miniGraph = synGraph.subgraph(miniSet)
    nodes = miniGraph.nodes()
    ss7 = []
    ssO = []
    if len(miniSet) == 3 or len(miniSet) == 2  :
       for node in nodes :
           to_append = [node,
                        str(miniGraph.node[node]['start']),
                        str(miniGraph.node[node]['sum']),
                        str(miniGraph.node[node]['contig']),
                        str(miniGraph.node[node]['max']),
                        str(miniGraph.node[node]['zeros']),
                        str(miniGraph.node[node]['total_num_counts']) 
                        ]
           if miniGraph.node[node]['contig'] == 'Super-Scaffold_7' :
               ss7.append(to_append)
           else:
               ssO.append(to_append)
       if len(ss7)==2 and len(ssO)==1:
           passed +=1 
           fout.write(','.join(ss7[0])+',')
           fout.write(','.join(ss7[1])+',')
           fout.write(','.join(ssO[0])+'\n')
           fout.write(','.join(ss7[1])+',')
           fout.write(','.join(ss7[0])+',')
           fout.write(','.join(ssO[0])+'\n')
       elif len(ss7) == 1 and len(ssO) ==1 :
           fout_single.write(','.join(ss7[0])+',')
           fout_single.write(','.join(ssO[0])+'\n')
       elif len(ss7) == 2 and len(ssO) ==0 :
           fout_single.write(','.join(ss7[0])+',')
           fout_single.write(','.join(ss7[1])+'\n')
           
           
       else:
           failed += 1 

fout.close() 
print(passed, failed)
               
               
        

479 0
